# client instance 생성

In [1]:
DEPLOYMENT_URL="http://10.8.1.229:8000/"
# DEPLOYMENT_URL="http://localhost:2024/"
# DEPLOYMENT_URL="http://localhost:8000/"

from langgraph_sdk import get_client

client = get_client(url=DEPLOYMENT_URL)

# thread 생성 (앨런의 channel에 대응)

- 생성 요청 후 thread_id를 포함한 데이터를 return 받음
- 하나의 thread에 대해서 여러 에이전트를 호출하고 대화가능
- 현재 앨런은 한개의 channel에 대해서 한개의 agent가 1:1로 대응되므로 현재는 고려할 필요 없지만, 추후에 1개의 채널에 대해서 여러 agent를 호출가능

In [2]:
def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data
 
 
def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)
 
        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    if msg_type == "ai":
                        return True
    return False

In [3]:
thread = await client.threads.create()
print(thread)

thread_id = thread["thread_id"]
print(thread_id)

{'thread_id': 'a3f9423e-ac3d-45f8-b641-d9423248172d', 'created_at': '2025-08-14T04:32:33.503151+00:00', 'updated_at': '2025-08-14T04:32:33.503151+00:00', 'metadata': {}, 'status': 'idle', 'config': {}, 'values': None, 'interrupts': {}, 'error': None}
a3f9423e-ac3d-45f8-b641-d9423248172d


# 에이전트 호출

- 해당 thread에서 어떤 agent를 호출할 지 assistant_id를 포함하여 요청을 보냄

## parameters
- thread_id : 위에서 생성한 thread_id
- assistant_id : 앨런의 persona_id에 대응, agent를 구분하는 id, 슬라이드 에이전트의 경우 "slide_generate_agent"
- input : 해당 agent 요청에 대한 parameter, 현재는 messages만 존재, 앨런 페이지에서 유저로 부터 받는 input을 해당 parameter에 dict로 넣어줌
- stream_mode : 어떤 event에 따라서 메시지를 받을지 지정, updates(한 단계가 끝날 때마나 메시지가 옴), events(llm이 생성하는 단위로 메시지 생성)

In [4]:
list_msg_id = list()


def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data


def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)

        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    msg_id = data['messages'][-1]['id']
                    if msg_id not in list_msg_id and msg_type == "ai":
                        list_msg_id.append(msg_id)
                        return True
    return False


def filter_silde_data(chunk):
    # slide 관련 데이터를 담은 메시지를 필터링하는 함수
    # 슬라이드 한페이지 단위로 메시지가 옴
    if 'updates' in chunk.event:
        node_name = chunk.data.keys()
        if 'executor' in node_name:
            return True

In [5]:
assistant_id = "slide_generate_agent"
list_chunk = list()

In [6]:
input = { "messages": "한국 반도체 현황. 모두 한글로 만들어줘"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    list_chunk.append(chunk)
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])
    # elif filter_silde_data(chunk):
    #     print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
    #     print(chunk)


##########  AI 메시지  ##########
한국 반도체 현황을 주제로 슬라이드를 생성하도록 하겠습니다.
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. 한국 반도체 산업의 역사와 발전 - 이 섹션은 한국 반도체 산업의 역사와 진화 과정을 조명하며, 산업 발전의 주요 사건과 주요 성취를 설명합니다. 
3. 한국 반도체 시장 점유율 분석 - 이 섹션에서는 한국 반도체 시장의 세계 시장 점유율과 주요 경쟁국과의 비교를 다룰 예정입니다. 
4. 한국 주요 반도체 기업의 동향 - 주요 한국 반도체 회사(삼성전자, SK하이닉스 등)의 현황 및 최근 동향을 다룹니다. 
5. 한국 반도체 기술과 혁신 동향 - 메모리 반도체와 비메모리 반도체를 포함한 한국의 최신 반도체 기술 동향에 대해 설명합니다. 
6. 한국 정부의 반도체 정책 - 한국 정부의 반도체산업 육성 전략과 정책지원 방안을 설명합니다. 

##########  AI 메시지  ##########

    슬라이드 생성에 필요한 조사를 시작합니다.
    
##########  AI 메시지  ##########

    슬라이드에 사용할 이미지를 검색하고 있습니다.
    
##########  AI 메시지  ##########

    슬라이드 디자인을 시작합니다.
    
##########  AI 메시지  ##########

    슬라이드를 생성하고 있습니다.
    
##########  AI 메시지  ##########
슬라이드 생성이 완료되었습니다.


In [7]:
for chunk in list_chunk:
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])


In [8]:
state = await client.threads.get_state(thread_id=thread_id)

for msg in state["values"]["messages"]:
    if msg["type"] != "human":
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        print(msg["content"])

##########  AI 메시지  ##########
한국 반도체 현황을 주제로 슬라이드를 생성하도록 하겠습니다.
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. 한국 반도체 산업의 역사와 발전 - 이 섹션은 한국 반도체 산업의 역사와 진화 과정을 조명하며, 산업 발전의 주요 사건과 주요 성취를 설명합니다. 
3. 한국 반도체 시장 점유율 분석 - 이 섹션에서는 한국 반도체 시장의 세계 시장 점유율과 주요 경쟁국과의 비교를 다룰 예정입니다. 
4. 한국 주요 반도체 기업의 동향 - 주요 한국 반도체 회사(삼성전자, SK하이닉스 등)의 현황 및 최근 동향을 다룹니다. 
5. 한국 반도체 기술과 혁신 동향 - 메모리 반도체와 비메모리 반도체를 포함한 한국의 최신 반도체 기술 동향에 대해 설명합니다. 
6. 한국 정부의 반도체 정책 - 한국 정부의 반도체산업 육성 전략과 정책지원 방안을 설명합니다. 

##########  AI 메시지  ##########
슬라이드 생성이 완료되었습니다.


In [9]:
len(state["values"]["slides"])

7

In [10]:
state["values"]["slides"]

[{'slide_type': 'title',
  'topic': '한국 반도체 현황',
  'idx': 0,
  'name': '타이틀',
  'description': '한국 반도체 현황에 대한 프레젠테이션 타이틀 페이지',
  'requirements': ['타이틀 텍스트', '부제목 또는 설명'],
  'research': True,
  'content': '## 타이틀\n\n"2023년 한국 반도체 산업 현황: 기회와 도전"\n\n### 배경\n\n2023년은 한국 반도체 산업에 있어 도전과 기회의 해였습니다. 주요 반도체 수출품인 메모리 반도체의 가격 하락으로 인해, 한국의 반도체 수출은 전년 대비 감소를 경험했습니다 [1]. 총 수출액은 1310억 9천만 달러로 집계되었지만, 이는 최근 5년간 가장 낮은 수치입니다 [2]. 이러한 상황에도 불구하고, 반도체 산업의 중요성은 지속적으로 증가하고 있습니다.\n\n### 성장 전망\n\n한국 반도체 산업은 세계 시장에서 계속해서 중요한 위치를 유지하고 있습니다. 향후 5년간 메모리 반도체의 수요가 회복될 것으로 전망되며, 한국은 2032년까지 글로벌 반도체 시장에서 20% 이상의 점유율을 기록할 것으로 예측됩니다 [3]. 특히, 새로운 기술과 생산 공정의 개발로 인해 반도체 제품의 다양성과 품질이 향상될 것으로 기대되고 있습니다.\n\n### 결론\n\n한국 반도체 산업은 2023년의 시장 변동에도 불구하고, 장기적으로 긍정적인 성장을 계속할 것입니다. 지속 가능한 기술 개발과 글로벌 시장 확대를 위한 전략적 투자가 필요합니다. 한국은 앞으로도 기술 리더십을 유지하며 반도체 산업의 중심에 서 있을 것입니다.\n\n### 출처\n\n[1] South Korea\'s Semiconductor Export Overview: https://koreatechtoday.com/south-koreas-semiconductor-giants-lead-amidst-tech-sector-shifts-a-look-back-at

In [11]:
for s in state["values"]["slides"]:
    with open(f"{s['idx']}.html", "w", encoding="utf-8") as f:
        f.write(s['html'])